In [43]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [44]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Pirâmide", None, None)
glfw.make_context_current(window)

In [45]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [46]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

In [47]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


In [48]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [49]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


In [50]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [51]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


In [52]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [53]:
# preparando espaço para 24 vértices usando 3 coordenadas (x,y,z)
vertices = np.zeros(16, [("position", np.float32, 3)])

In [54]:
# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    # Face 1 da Pirâmide
    (-0.5,  0.0, +0.5),
    (+0.5,  0.0, +0.5),
    ( 0.0, +1.0, +0.0),

    # Face 2 da Pirâmide
    (+0.5,  0.0, +0.5),
    (+0.5,  0.0, -0.5),         
    (+0.0, +1.0, +0.0),
    
    # Face 3 da Pirâmide
    (+0.5,  0.0, -0.5),
    (-0.5,  0.0, -0.5),            
    (+0.0, +1.0,  0.0),

    # Face 4 da Pirâmide
    (-0.5,  0.0, -0.5),
    (-0.5,  0.0, +0.5),         
    ( 0.0, +1.0,  0.0),
    
    # Face base da Pirâmide
    (+0.5,  0.0, -0.5),
    (-0.5,  0.0, -0.5),
    (+0.5,  0.0, +0.5),            
    (-0.5,  0.0, +0.5)
    
]

In [55]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


In [56]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [57]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


In [58]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [59]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

In [60]:
loc_color = glGetUniformLocation(program, "color")

In [61]:

s_x = 1.0
s_y = 1.0
s_z = 1.0

t_x = 0.0
t_y = 0.0
t_z = 0.0

stopRotation = 0

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, t_z, s_x, s_y, s_z, stopRotation
    if key == 87: t_y += 0.01 #cima W
    if key == 83: t_y -= 0.01 #baixo S
    if key == 65: t_x -= 0.01 #esquerda A
    if key == 68: t_x += 0.01 #direita D
    if key == 69: t_z += 0.01 #eixo z (tecla E)
    if key == 81: t_z -= 0.01 #eixo z (tecla Q)


    if key == 263: # seta esquerda
        s_x += 0.01 
        s_y += 0.01
        s_z += 0.01
    if key == 262: # seta direita
        s_x -= 0.01 
        s_y -= 0.01
        s_z -= 0.01
    
    if key == 32: 
        if stopRotation == 0: 
            stopRotation = 1
        
glfw.set_key_callback(window,key_event)

In [62]:
glfw.show_window(window)

In [63]:
import math
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

from numpy import random


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    ### apenas para visualizarmos o cubo rotacionando
    if stopRotation == 0:
        d += 0.0001 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.matrix([    [cos_d, -sin_d, 0.0, 0.0], 
                                    [sin_d,  cos_d, 0.0, 0.0], 
                                    [0.0,      0.0, 1.0, 0.0], 
                                    [0.0,      0.0, 0.0, 1.0]], np.float32)
    
    mat_rotation_x = np.matrix([    [1.0,   0.0,    0.0, 0.0], 
                                    [0.0, cos_d, -sin_d, 0.0], 
                                    [0.0, sin_d,  cos_d, 0.0], 
                                    [0.0,   0.0,    0.0, 1.0]], np.float32)
    
    mat_rotation_y = np.matrix([    [cos_d,  0.0, sin_d, 0.0], 
                                    [0.0,    1.0,   0.0, 0.0], 
                                    [-sin_d, 0.0, cos_d, 0.0], 
                                    [0.0,    0.0,   0.0, 1.0]], np.float32)
    
    mat_translation = np.matrix([   [1.0, 0.0, 0.0, t_x], 
                                    [0.0, 1.0, 0.0, t_y], 
                                    [0.0, 0.0, 1.0, t_z], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)

    mat_scale = np.matrix([   [s_x, 0.0, 0.0, 0.0], 
                              [0.0, s_y, 0.0, 0.0], 
                              [0.0, 0.0, s_z, 0.0], 
                              [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    mat_rotation = mat_rotation_x @ mat_rotation_y @ mat_rotation_z
    mat_transform = mat_translation @ mat_scale @ mat_rotation



    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
   
    
    glUniform4f(loc_color, 0.643, 0.290, 0.247, 1.0) ### vermelho escuro
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 3)
    
    glUniform4f(loc_color, 0.945, 0.611, 0.474, 1.0) ### vermelho claro
    glDrawArrays(GL_TRIANGLE_STRIP, 3, 3)
    
    glUniform4f(loc_color, 0.796, 0.874, 0.741, 1.0) ### verde menta
    glDrawArrays(GL_TRIANGLE_STRIP, 6, 3)
    
    glUniform4f(loc_color, 0.964, 0.956, 0.823, 1.0) ### amarelo claro
    glDrawArrays(GL_TRIANGLE_STRIP, 9, 3)
    
    glUniform4f(loc_color, 0.831, 0.878, 0.607, 1.0) ### verde
    glDrawArrays(GL_TRIANGLE_STRIP, 12, 4)
    
    
    glfw.swap_buffers(window)

glfw.terminate()